<a href="https://colab.research.google.com/github/ogwustephen2023/Paging-Mission/blob/main/Paging_Mission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import json
from datetime import datetime, timedelta


def parse_line(line):
    fields = line.strip().split('|')
    timestamp = datetime.strptime(fields[0], "%Y%m%d %H:%M:%S.%f")
    satellite_id = int(fields[1])
    red_high_limit = float(fields[2])
    yellow_high_limit = float(fields[3])
    yellow_low_limit = float(fields[4])
    red_low_limit = float(fields[5])
    raw_value = float(fields[6])
    component = fields[7]
    return (timestamp, satellite_id, red_high_limit, yellow_high_limit, yellow_low_limit, red_low_limit, raw_value, component)

def check_violations(readings, limit, condition):
    count = 0
    for timestamp, value in readings:
        if condition(value, limit):
            count += 1
        if count >= 3:
            return timestamp
    return None

def generate_alerts(filename):
    data = {}

    with open(filename, 'r') as file:
        for line in file:
            timestamp, satellite_id, red_high_limit, yellow_high_limit, yellow_low_limit, red_low_limit, raw_value, component = parse_line(line)
            if satellite_id not in data:
                data[satellite_id] = {'BATT': [], 'TSTAT': []}
            data[satellite_id][component].append((timestamp, raw_value, red_high_limit, red_low_limit))

    alerts = []
    for satellite_id, components in data.items():
        for component, readings in components.items():
            # Sort readings by timestamp
            readings.sort()
            timestamps, values, red_high_limits, red_low_limits = zip(*readings)

            # Check red high limit violation for TSTAT
            if component == 'TSTAT':
                red_high_limit = red_high_limits[0]  # Same limit for all readings
                violation_timestamp = check_violations(list(zip(timestamps, values)), red_high_limit, lambda v, limit: v > limit)
                if violation_timestamp:
                    alerts.append({
                        "satelliteId": satellite_id,
                        "severity": "RED HIGH",
                        "component": component,
                        "timestamp": violation_timestamp.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
                    })

            # Check red low limit violation for BATT
            if component == 'BATT':
                red_low_limit = red_low_limits[0]  # Same limit for all readings
                violation_timestamp = check_violations(list(zip(timestamps, values)), red_low_limit, lambda v, limit: v < limit)
                if violation_timestamp:
                    alerts.append({
                        "satelliteId": satellite_id,
                        "severity": "RED LOW",
                        "component": component,
                        "timestamp": violation_timestamp.strftime("%Y-%m-%dT%H:%M:%S.%fZ")
                    })

    return alerts

# Sample data input
input_data = """20180101 23:01:05.001|1001|101|98|25|20|99.9|TSTAT
20180101 23:01:09.521|1000|17|15|9|8|7.8|BATT
20180101 23:01:26.011|1001|101|98|25|20|99.8|TSTAT
20180101 23:01:38.001|1000|101|98|25|20|102.9|TSTAT
20180101 23:01:49.021|1000|101|98|25|20|87.9|TSTAT
20180101 23:02:09.014|1001|101|98|25|20|89.3|TSTAT
20180101 23:02:10.021|1001|101|98|25|20|89.4|TSTAT
20180101 23:02:11.302|1000|17|15|9|8|7.7|BATT
20180101 23:03:03.008|1000|101|98|25|20|102.7|TSTAT
20180101 23:03:05.009|1000|101|98|25|20|101.2|TSTAT
20180101 23:04:06.017|1001|101|98|25|20|89.9|TSTAT
20180101 23:04:11.531|1000|17|15|9|8|7.9|BATT
20180101 23:05:05.021|1001|101|98|25|20|89.9|TSTAT
20180101 23:05:07.421|1001|17|15|9|8|7.9|BATT
"""

# Write sample data to file for testing
with open('sample_data.txt', 'w') as f:
    f.write(input_data)

# Generate alerts
alerts = generate_alerts('sample_data.txt')

# Print alerts in JSON format
print(json.dumps(alerts, indent=4))


[
    {
        "satelliteId": 1000,
        "severity": "RED LOW",
        "component": "BATT",
        "timestamp": "2018-01-01T23:04:11.531000Z"
    },
    {
        "satelliteId": 1000,
        "severity": "RED HIGH",
        "component": "TSTAT",
        "timestamp": "2018-01-01T23:03:05.009000Z"
    }
]
